# SUPERVISED LEARNING


In [0]:
import numpy as np 
import pandas as pd

import math 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
%matplotlib inline
import os
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
os.listdir('/content/drive/My Drive/Python new/PYTHON/PROJECT/PROJECT 1/')

['.ipynb_checkpoints',
 'Consumer_Complaints_test_share.csv',
 'Consumer_Complaints_train.csv',
 'Consumer_Complaints.ipynb']

In [0]:
cc_train=pd.read_csv(r'/content/drive/My Drive/Python new/PYTHON/PROJECT/PROJECT 1/Consumer_Complaints_train.csv')

cc_test=pd.read_csv(r'/content/drive/My Drive/Python new/PYTHON/PROJECT/PROJECT 1/Consumer_Complaints_test_share.csv')

In [0]:
cc_test["data"]='test'

In [6]:
cc_test.shape

(119606, 18)

Filling target variable of Test datasets with None for neccessary preprocessing

In [0]:
cc_test["Consumer disputed?"]=None

In [0]:
cc_train['data']='train'

In [9]:
cc_train.shape

(478421, 19)

Combining Train and Test data

In [10]:
cc=pd.concat([cc_train,cc_test],0,ignore_index=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [11]:
cc.shape

(598027, 19)

In [12]:
cc["Consumer disputed?"].value_counts()

No     376990
Yes    101431
Name: Consumer disputed?, dtype: int64

Checking features 

In [13]:
cc.columns

Index(['Company', 'Company public response', 'Company response to consumer',
       'Complaint ID', 'Consumer complaint narrative',
       'Consumer consent provided?', 'Consumer disputed?', 'Date received',
       'Date sent to company', 'Issue', 'Product', 'State', 'Sub-issue',
       'Sub-product', 'Submitted via', 'Tags', 'Timely response?', 'ZIP code',
       'data'],
      dtype='object')

In [14]:
cc.isna().sum()

Company                              0
Company public response         484859
Company response to consumer         0
Complaint ID                         0
Consumer complaint narrative    504376
Consumer consent provided?      428676
Consumer disputed?              119606
Date received                        0
Date sent to company                 0
Issue                                0
Product                              0
State                             4764
Sub-issue                       365685
Sub-product                     173225
Submitted via                        1
Tags                            513950
Timely response?                     0
ZIP code                          4774
data                                 0
dtype: int64

In [15]:
cc.dtypes

Company                         object
Company public response         object
Company response to consumer    object
Complaint ID                     int64
Consumer complaint narrative    object
Consumer consent provided?      object
Consumer disputed?              object
Date received                   object
Date sent to company            object
Issue                           object
Product                         object
State                           object
Sub-issue                       object
Sub-product                     object
Submitted via                   object
Tags                            object
Timely response?                object
ZIP code                        object
data                            object
dtype: object

In [0]:
for col in ['Sub-product','Sub-issue','Consumer complaint narrative',
            'Company public response','Tags','Consumer consent provided?']:
    varname=col.replace('-','_').replace('?','').replace(" ",'_')+'_isNan'
    cc[varname]=np.where(pd.isnull(cc[col]),1,0)
    cc.drop([col],1,inplace=True)
    

In [0]:
cc['Consumer disputed?']= np.where(cc['Consumer disputed?']=='Yes',1,0)
cc['Timely response?']=np.where(cc['Timely response?']=='Yes',1,0)

In [18]:
cc['Company response to consumer'].value_counts()

Closed with explanation            442756
Closed with non-monetary relief     76861
Closed with monetary relief         41188
Closed without relief               17769
Closed                              14227
Closed with relief                   5224
Untimely response                       2
Name: Company response to consumer, dtype: int64

In [19]:
cc.groupby('Company response to consumer')['Consumer disputed?'].mean()

Company response to consumer
Closed                             0.179518
Closed with explanation            0.186990
Closed with monetary relief        0.090633
Closed with non-monetary relief    0.103564
Closed with relief                 0.107198
Closed without relief              0.215656
Untimely response                  0.500000
Name: Consumer disputed?, dtype: float64

Creating Dummies

In [0]:
lp_dummies=pd.get_dummies(cc['Company response to consumer'])

In [0]:
cc=pd.concat([lp_dummies,cc],1)

In [0]:
cc=cc.drop(['Company response to consumer'],1)

In [23]:
cc

,Closed,Closed with explanation,Closed with monetary relief,Closed with non-monetary relief,Closed with relief,Closed without relief,Untimely response,Company,Complaint ID,Consumer disputed?,Date received,Date sent to company,Issue,Product,State,Submitted via,Timely response?,ZIP code,data,Sub_product_isNan,Sub_issue_isNan,Consumer_complaint_narrative_isNan,Company_public_response_isNan,Tags_isNan,Consumer_consent_provided_isNan
0,0,1,0,0,0,0,0,Wells Fargo & Company,856103,0,2014-05-15,2014-05-16,Billing statement,Credit card,MI,Web,1,48342,train,1,1,1,1,0,1
1,1,0,0,0,0,0,0,Santander Bank US,1034666,0,2014-09-18,2014-09-24,"Making/receiving payments, sending money",Bank account or service,PA,Referral,1,18042,train,0,1,1,1,1,1
2,0,0,0,1,0,0,0,Equifax,756363,0,2014-03-13,2014-04-03,Incorrect information on credit report,Credit reporting,CA,Referral,1,92427,train,1,0,1,1,1,1
3,0,0,1,0,0,0,0,U.S. Bancorp,1474177,0,2015-07-17,2015-07-17,Billing statement,Credit card,GA,Web,1,305XX,train,1,1,0,0,0,0
4,0,1,0,0,0,0,0,Bank of America,1132572,0,2014-11-20,2014-11-28,Transaction issue,Credit card,MA,Web,1,02127,train,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598022,0,0,1,0,0,0,0,JPMorgan Chase & Co.,153482,0,2012-09-15,2012-09-21,Closing/Cancelling account,Credit card,IL,Web,1,60538,test,1,1,1,1,1,1
598023,0,1,0,0,0,0,0,Experian,2023523,0,2016-07-21,2016-07-21,Credit reporting company's investigation,Credit reporting,CA,Web,1,956XX,test,1,0,0,0,1,0
598024,0,1,0,0,0,0,0,"FAIR COLLECTIONS & OUTSOURCING, INC.",1413678,0,2015-06-09,2015-06-12,Disclosure verification of debt,Debt collection,CA,Phone,1,90803,test,0,0,1,0,0,1
598025,0,1,0,0,0,0,0,Encore Capital Group,732458,0,2014-02-26,2014-02-27,Disclosure verification of debt,Debt collection,PA,Phone,1,17764,test,0,0,1,1,1,1


In [24]:
cc['Product'].value_counts()

Mortgage                   195079
Debt collection            108047
Credit reporting           101612
Credit card                 71613
Bank account or service     68094
Consumer Loan               23263
Student loan                18635
Money transfers              4176
Payday loan                  4048
Prepaid card                 2813
Other financial service       635
Virtual currency               12
Name: Product, dtype: int64

In [25]:
cc.groupby('Product')['Consumer disputed?'].mean()

Product
Bank account or service    0.162540
Consumer Loan              0.189528
Credit card                0.174116
Credit reporting           0.141538
Debt collection            0.157154
Money transfers            0.119013
Mortgage                   0.192045
Other financial service    0.184252
Payday loan                0.136364
Prepaid card               0.119445
Student loan               0.169251
Virtual currency           0.250000
Name: Consumer disputed?, dtype: float64

In [0]:
#print(pd.describe_option())

#pd.options.display.max_rows=100

###pd.set_option('min_rows',100)

In [0]:
k=cc['Issue'].value_counts()
for val in k.axes[0][0:10]:
    varname='Issue_'+val.replace(',','_').replace(' ','_')
    cc[varname]=np.where(cc['Issue']==val,1,0)
del cc['Issue']

In [27]:
cc

,Closed,Closed with explanation,Closed with monetary relief,Closed with non-monetary relief,Closed with relief,Closed without relief,Untimely response,Company,Complaint ID,Consumer disputed?,Date received,Date sent to company,Product,State,Submitted via,Timely response?,ZIP code,data,Sub_product_isNan,Sub_issue_isNan,Consumer_complaint_narrative_isNan,Company_public_response_isNan,Tags_isNan,Consumer_consent_provided_isNan,Issue_Loan_modification_collection_foreclosure,Issue_Incorrect_information_on_credit_report,Issue_Loan_servicing__payments__escrow_account,Issue_Cont'd_attempts_collect_debt_not_owed,Issue_Account_opening__closing__or_management,Issue_Disclosure_verification_of_debt,Issue_Communication_tactics,Issue_Deposits_and_withdrawals,Issue_Application__originator__mortgage_broker,Issue_Billing_disputes
0,0,1,0,0,0,0,0,Wells Fargo & Company,856103,0,2014-05-15,2014-05-16,Credit card,MI,Web,1,48342,train,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,Santander Bank US,1034666,0,2014-09-18,2014-09-24,Bank account or service,PA,Referral,1,18042,train,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,Equifax,756363,0,2014-03-13,2014-04-03,Credit reporting,CA,Referral,1,92427,train,1,0,1,1,1,1,0,1,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,U.S. Bancorp,1474177,0,2015-07-17,2015-07-17,Credit card,GA,Web,1,305XX,train,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,Bank of America,1132572,0,2014-11-20,2014-11-28,Credit card,MA,Web,1,02127,train,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598022,0,0,1,0,0,0,0,JPMorgan Chase & Co.,153482,0,2012-09-15,2012-09-21,Credit card,IL,Web,1,60538,test,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0
598023,0,1,0,0,0,0,0,Experian,2023523,0,2016-07-21,2016-07-21,Credit reporting,CA,Web,1,956XX,test,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
598024,0,1,0,0,0,0,0,"FAIR COLLECTIONS & OUTSOURCING, INC.",1413678,0,2015-06-09,2015-06-12,Debt collection,CA,Phone,1,90803,test,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0
598025,0,1,0,0,0,0,0,Encore Capital Group,732458,0,2014-02-26,2014-02-27,Debt collection,PA,Phone,1,17764,test,0,0,1,1,1,1,0,0,0,0,0,1,0,0,0,0


In [28]:
cc.isna().sum()

Closed                                               0
Closed with explanation                              0
Closed with monetary relief                          0
Closed with non-monetary relief                      0
Closed with relief                                   0
Closed without relief                                0
Untimely response                                    0
Company                                              0
Complaint ID                                         0
Consumer disputed?                                   0
Date received                                        0
Date sent to company                                 0
Product                                              0
State                                             4764
Submitted via                                        1
Timely response?                                     0
ZIP code                                          4774
data                                                 0
Sub_produc

Dropping columns which having very less impact on response variable.

In [0]:
cc=cc.drop(['ZIP code'],1)

In [30]:
cc.columns

Index(['Closed', 'Closed with explanation', 'Closed with monetary relief',
       'Closed with non-monetary relief', 'Closed with relief',
       'Closed without relief', 'Untimely response', 'Company', 'Complaint ID',
       'Consumer disputed?', 'Date received', 'Date sent to company',
       'Product', 'State', 'Submitted via', 'Timely response?', 'data',
       'Sub_product_isNan', 'Sub_issue_isNan',
       'Consumer_complaint_narrative_isNan', 'Company_public_response_isNan',
       'Tags_isNan', 'Consumer_consent_provided_isNan',
       'Issue_Loan_modification_collection_foreclosure',
       'Issue_Incorrect_information_on_credit_report',
       'Issue_Loan_servicing__payments__escrow_account',
       'Issue_Cont'd_attempts_collect_debt_not_owed',
       'Issue_Account_opening__closing__or_management',
       'Issue_Disclosure_verification_of_debt', 'Issue_Communication_tactics',
       'Issue_Deposits_and_withdrawals',
       'Issue_Application__originator__mortgage_broker',
  

In [0]:
feat=['Company','Product','Submitted via','State']
for tr in feat:
    f=cc[tr].value_counts()
    cat=f.index[f>10000]
    for t in cat:
        name = tr +'_'+t
        cc[name]=(cc[tr]==t).astype(int)
    del cc[tr]

In [0]:
#pd.options.display.min_rows=70
#pd.options.display.max_rows=100


Converting Date and time into useful feature.

In [0]:
from datetime import date

In [0]:
cc["Days"]=pd.to_datetime(cc['Date sent to company'])-pd.to_datetime(cc['Date received'])

In [34]:
cc['Days']

0         1 days
1         6 days
2        21 days
3         0 days
4         8 days
           ...  
598022    6 days
598023    0 days
598024    3 days
598025    1 days
598026    0 days
Name: Days, Length: 598027, dtype: timedelta64[ns]

In [35]:
cc["Days"].value_counts()

0 days      262268
1 days       73537
2 days       47613
3 days       38469
4 days       35760
             ...  
482 days         1
293 days         1
312 days         1
520 days         1
340 days         1
Name: Days, Length: 420, dtype: int64

In [0]:
cc['Days']=cc["Days"].astype(str).map(lambda x: x.rstrip('days 00:00:00.000000000').rstrip(' days +'))
cc["Days"]=cc["Days"].map(lambda x: x.lstrip())

In [37]:
v=cc["Days"]
v.value_counts()

       262268
1       76698
2       49568
3       38973
4       36162
        ...  
392         1
591         1
713         1
414         1
588         1
Name: Days, Length: 378, dtype: int64

In [0]:
cc["Days"]=cc['Days'].replace(to_replace=[''],value=[0])


In [0]:
cc["Days"]=[int(x) for x in cc["Days"]]
cc["Days"]=cc['Days'].replace(to_replace=[-1],value=[0])

In [0]:
cc=cc.drop(["Complaint ID","Date sent to company","Date received"],1)

In [0]:
cc["Days"]=pd.to_numeric(cc["Days"],errors="coerce")

In [42]:
cc.dtypes

Closed                             uint8
Closed with explanation            uint8
Closed with monetary relief        uint8
Closed with non-monetary relief    uint8
Closed with relief                 uint8
                                   ...  
State_AZ                           int64
State_WA                           int64
State_MA                           int64
State_CO                           int64
Days                               int64
Length: 67, dtype: object

In [43]:
cc_tra=cc[cc["data"]=="train"]
cc_tra.shape

(478421, 67)

In [44]:
cc_tes=cc[cc["data"]=="test"]
cc_tes.shape

(119606, 67)

In [45]:
cc_tes=cc_tes.drop(["Consumer disputed?"],1)
cc_tes.shape

(119606, 66)

In [0]:
cc_tra=cc_tra.drop(["data"],1)
cc_tes=cc_tes.drop(["data"],1)

In [47]:
cc.isna().sum()

Closed                             0
Closed with explanation            0
Closed with monetary relief        0
Closed with non-monetary relief    0
Closed with relief                 0
                                  ..
State_AZ                           0
State_WA                           0
State_MA                           0
State_CO                           0
Days                               0
Length: 67, dtype: int64

In [0]:
cc_tra1,cc_tra2=train_test_split(cc_tra,test_size=0.2,random_state=2)

In [49]:
cc_tra1.shape

(382736, 66)

In [0]:
x_train=cc_tra1.drop(['Consumer disputed?'],1)

y_train=cc_tra1["Consumer disputed?"]

x_test=cc_tra2.drop(['Consumer disputed?'],1)

y_test=cc_tra2['Consumer disputed?']

In [51]:
x_train.shape

(382736, 65)

# Logistics Regression

In [0]:
logr=LogisticRegression(penalty='l2',class_weight="balanced",random_state= 3)

In [53]:
logr.fit(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=3, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [54]:
roc_auc_score(y_test,logr.predict(x_test))#l2 score 

0.5834170420087239

In [55]:
roc_auc_score(y_test,logr.predict(x_test))

0.5834170420087239

# RandomForest 

In [0]:
from sklearn.model_selection import RandomizedSearchCV

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [0]:
clf=RandomForestClassifier()

In [0]:
# RandomSearchCV accept parameters values as dictionaries.
# In example given below we have constructed dictionary for 
#different parameter values that we want to
# try for randomForest model

params={
    'n_estimators':[1000,3000,4000],
    'criterion':['gini','entropy'],
    'max_depth':[None,2,5,7,9],
    'min_samples_split':[10,20,30,40],
    'min_samples_leaf':[10,30,40,50],
    'max_features':[None,2,3,5,7],
    'bootstrap':[True,False],
     'verbose':[1],
    'class_weight':[None,'balanced']}

In [0]:
random_search=RandomizedSearchCV(clf,param_distributions=params,n_iter=3,scoring='roc_auc',cv=5)

In [61]:
random_search.fit(x_train,y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   34.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   35.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   35.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [0]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [63]:
report(random_search.cv_results_,5)

Model with rank: 1
Mean validation score: 0.614 (std: 0.00249)
Parameters: {'verbose': 1, 'n_estimators': 4000, 'min_samples_split': 30, 'min_samples_leaf': 50, 'max_features': None, 'max_depth': 5, 'criterion': 'gini', 'class_weight': 'balanced', 'bootstrap': True}

Model with rank: 2
Mean validation score: 0.609 (std: 0.00236)
Parameters: {'verbose': 1, 'n_estimators': 3000, 'min_samples_split': 10, 'min_samples_leaf': 40, 'max_features': 7, 'max_depth': 5, 'criterion': 'gini', 'class_weight': 'balanced', 'bootstrap': False}

Model with rank: 3
Mean validation score: 0.601 (std: 0.00304)
Parameters: {'verbose': 1, 'n_estimators': 1000, 'min_samples_split': 20, 'min_samples_leaf': 30, 'max_features': 2, 'max_depth': 2, 'criterion': 'gini', 'class_weight': None, 'bootstrap': False}



In [0]:
# select the best values from results above, they will vary slightly with each run
rf=RandomForestClassifier(verbose= 1, n_estimators = 4000, min_samples_split= 30, min_samples_leaf= 50, max_features = None, 
                          max_depth = 5, criterion = 'gini', class_weight = 'balanced', bootstrap = True
)

In [66]:
rf.fit(x_train,y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 4000 out of 4000 | elapsed: 47.5min finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=5, max_features=None,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=50, min_samples_split=30,
                       min_weight_fraction_leaf=0.0, n_estimators=4000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=1, warm_start=False)

In [69]:
ypred2=rf.predict(x_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 4000 out of 4000 | elapsed:   21.3s finished


In [71]:
roc_auc_score(y_test,ypred2)

0.581746158066147

# Feature Importance

In [72]:
feat_imp_df=pd.DataFrame({'features':x_train.columns,'importance':rf.feature_importances_})

feat_imp_df.sort_values('importance',ascending=False)

,features,importance
1,Closed with explanation,3.187394e-01
43,Submitted via_Web,1.979412e-01
9,Sub_issue_isNan,1.368960e-01
5,Closed without relief,1.229239e-01
0,Closed,6.162876e-02
...,...,...
29,"Company_TransUnion Intermediate Holdings, Inc.",1.884361e-06
33,Company_Nationstar Mortgage,1.730486e-06
35,Company_U.S. Bancorp,1.044193e-06
4,Closed with relief,6.169964e-07
